In [ ]:
product_sentiment = df_reviews.groupby("product")["sentiment_score"].mean().sort_values()
product_sentiment.plot(kind="barh", title="Average Sentiment by Product")
plt.xlabel("Sentiment Score")
plt.tight_layout()
plt.show()

In [ ]:
shipment_df = shipment_counts.to_pandas()
shipment_df.plot(x="shipping_date", y="shipment_count", kind="line", title="Shipments Per Day")
plt.ylabel("Number of Shipments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
low_volume_days = shipment_df[shipment_df["shipment_count"] < 5]
print("Low-volume shipping days:\n", low_volume_days)

In [ ]:
carrier_counts = df_shipping.groupby("carrier")["order_id"].count()
carrier_counts.plot(kind="bar", title="Total Shipments by Carrier")
plt.ylabel("Number of Shipments")
plt.tight_layout()
plt.show()

In [ ]:
# Assume both dataframes are loaded: reviews_df and shipping_df
combined_df = reviews_df.merge(shipping_df, on="order_id", how="inner")
combined_df.head()

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col

# Step 1: Define connection parameters
connection_parameters = {
    "account": "your_account_id",
    "user": "your_username",
    "password": "your_password",
    "role": "your_user_role",
    "warehouse": "your_virtual_warehouse",
    "database": "AVALANCHE_DB",
    "schema": "AVALANCHE_SCHEMA"
}

# Step 2: Create a session
session = Session.builder.configs(connection_parameters).create()

# Step 3: Save the merged DataFrame to a new table in Snowflake
combined_df.write.save_as_table("COMBINED_REVIEWS_SHIPPING", mode="overwrite")

# Step 4: Validate that the new table exists and looks correct
session.table("COMBINED_REVIEWS_SHIPPING").limit(5).show()

In [ ]:
import matplotlib.pyplot as plt
avg_sentiment = (    combined_df.groupby("status")["sentiment_score"]
    .mean()
    .sort_values()
)

avg_sentiment.plot(kind="barh", title="Avg Sentiment by Shipping Status")
plt.xlabel("Sentiment Score")
plt.ylabel("Shipping Status")
plt.tight_layout()
plt.show()